In [1]:
import numpy as np


In [2]:
learning_rate = 1
input_size = 2
hidden_size = 16
output_size = 1
Wxh = np.random.randn(hidden_size,input_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(output_size,hidden_size) * 0.01


In [3]:
binary_dim = 8
largest_num = np.power(2,binary_dim)

In [4]:
np.array([[1]]).shape

(1, 1)

In [5]:
int2binary = {}
binary = np.unpackbits(np.array([range(largest_num)],dtype=np.uint8).T,axis=1)
for i in range(largest_num):
    int2binary[i] = binary[i]

In [6]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output


In [7]:
np.array([[1],[2]]).shape


(2, 1)

In [8]:
def lossFun(a, b, c, hprev):
    xs, ms, hs, ys, ps, y = {}, {}, {}, {}, {}, {}
    loss = 0
    hs[-1] = np.copy(hprev)
    for t in range(binary_dim):
        xs[t] = np.array([a[[binary_dim-t-1]],b[[binary_dim-t-1]]])
        ms[t] = np.dot(Wxh,xs[t])+np.dot(Whh,hs[t-1])
        hs[t] = sigmoid(ms[t])
        ys[t] = np.dot(Why, hs[t])
        ps[t] = sigmoid(ys[t])
        y[t] = np.array([c[binary_dim-t-1]]).T
        loss += np.abs(y[t] - ps[t])
    
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh),np.zeros_like(Why)
    for t in reversed(range(binary_dim)):
        d1 = ps[t]*(ps[t]-1)
        dWhy += np.dot(d1,hs[t].T)
        d2 = np.dot(Why, ms[t])
        d3 = np.dot(d2,(1-ms[t]).T)
        dWhh += np.dot(d3,hs[t-1])
        dWxh += np.dot(d3.T,xs[t].T)
    
    #for dparam in [dWxh, dWhh, dWhy]:
     #   np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, hs[binary_dim-1]
        

In [9]:
def add_num(int_a,int_b,int_c,h):
    a = int2binary[int_a]
    b = int2binary[int_b]
    c = int2binary[int_c]
    d = np.zeros_like(c)
      
    for t in range(binary_dim):
        x = np.array([a[[binary_dim-t-1]],b[[binary_dim-t-1]]])
        m = np.dot(Wxh,x)+np.dot(Whh,h)
        h = sigmoid(m)
        ys = np.dot(Why, h)
        p = sigmoid(ys)
        #print((p[0][0]))
        #d[binary_dim-t-1] = np.round(p[0][0])
        if(p[0][0]>=0.5):
            d[binary_dim-t-1] = 1
        else:
            d[binary_dim-t-1] = 0
        #print(type(p))
        #out.append(cl)
    out=0
    for index,x in enumerate(reversed(d)):
        out += x*pow(2,index)
    
    print("a: ",a)
    print("b: ",b)
    print("pred: ",str(d))
    print("true: ",c)
    print(str(int_a)+" + " +str(int_b)+ " = "+ str(out))
    
    
    
        

In [10]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
smooth_loss = -np.log(1.0/input_size)*binary_dim
n = 0
while n <= 100*1000:
    
    if n==0:
        hprev = np.zeros((hidden_size,1))
        
    int_a = np.random.randint(largest_num/2)
    a = int2binary[int_a]
    int_b = np.random.randint(largest_num/2)
    b = int2binary[int_b]
    int_c = int_a + int_b
    c = int2binary[int_c]
    loss, dWxh, dWhh, dWhy, hprev = lossFun(a,b,c,hprev)
    
    smooth_loss = 0.999*smooth_loss + 0.001*loss
    if n%1000 == 0:
        print('iter: ',n)
        print('smooth_loss: ',smooth_loss)
        add_num(int_a,int_b,int_c,hprev)
        
    for param, dparam, mem in zip([Wxh, Whh, Why],
                                  [dWxh, dWhh, dWhy],
                                  [mWxh, mWhh, mWhy]):
        mem += dparam*dparam
        param += (-learning_rate) * (1/(np.sqrt(mem + 1e-8)))
    n+=1

iter:  0
smooth_loss:  [[5.54363233]]
a:  [0 1 0 1 1 1 1 0]
b:  [0 0 1 1 0 1 0 1]
pred:  [0 0 0 0 0 0 0 0]
true:  [1 0 0 1 0 0 1 1]
94 + 53 = 0


c:\users\a.ezhilarasan\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


iter:  1000
smooth_loss:  [[4.62769261]]
a:  [0 1 1 1 1 0 1 1]
b:  [0 0 0 1 0 0 1 1]
pred:  [0 1 1 1 1 0 1 1]
true:  [1 0 0 0 1 1 1 0]
123 + 19 = 123
iter:  2000
smooth_loss:  [[4.2950932]]
a:  [0 1 0 0 0 1 0 0]
b:  [0 0 1 0 0 1 1 0]
pred:  [0 1 1 1 0 1 1 1]
true:  [0 1 1 0 1 0 1 0]
68 + 38 = 119
iter:  3000
smooth_loss:  [[4.17590047]]
a:  [0 1 1 1 1 0 0 1]
b:  [0 0 1 1 1 1 1 1]
pred:  [0 1 1 1 1 1 1 1]
true:  [1 0 1 1 1 0 0 0]
121 + 63 = 127
iter:  4000
smooth_loss:  [[4.12985135]]
a:  [0 0 0 0 0 1 0 0]
b:  [0 1 0 1 1 1 1 1]
pred:  [0 1 0 1 1 1 1 1]
true:  [0 1 1 0 0 0 1 1]
4 + 95 = 95
iter:  5000
smooth_loss:  [[4.1163613]]
a:  [0 1 1 0 1 0 1 0]
b:  [0 1 1 0 1 0 0 1]
pred:  [0 1 1 0 1 0 1 1]
true:  [1 1 0 1 0 0 1 1]
106 + 105 = 107
iter:  6000
smooth_loss:  [[4.0921071]]
a:  [0 0 0 1 0 1 1 1]
b:  [0 0 1 1 1 1 0 0]
pred:  [1 0 1 1 1 1 1 1]
true:  [0 1 0 1 0 0 1 1]
23 + 60 = 191
iter:  7000
smooth_loss:  [[4.10894958]]
a:  [0 1 0 1 1 0 0 1]
b:  [0 0 1 0 0 1 1 1]
pred:  [0 1 1 1 1 1 1 

iter:  56000
smooth_loss:  [[4.07591635]]
a:  [0 0 1 1 1 0 0 1]
b:  [0 1 1 0 1 0 0 0]
pred:  [0 1 1 1 1 1 0 1]
true:  [1 0 1 0 0 0 0 1]
57 + 104 = 125
iter:  57000
smooth_loss:  [[4.06337981]]
a:  [0 1 1 0 0 0 1 0]
b:  [0 1 0 1 1 0 1 1]
pred:  [0 1 1 1 1 0 1 1]
true:  [1 0 1 1 1 1 0 1]
98 + 91 = 123
iter:  58000
smooth_loss:  [[4.07701161]]
a:  [0 1 1 0 0 0 0 1]
b:  [0 0 0 1 0 1 1 0]
pred:  [0 1 1 1 0 1 1 1]
true:  [0 1 1 1 0 1 1 1]
97 + 22 = 119
iter:  59000
smooth_loss:  [[4.03684872]]
a:  [0 1 0 1 0 1 0 1]
b:  [0 0 0 0 0 0 1 1]
pred:  [0 1 0 1 0 1 1 1]
true:  [0 1 0 1 1 0 0 0]
85 + 3 = 87
iter:  60000
smooth_loss:  [[4.0483648]]
a:  [0 1 1 0 0 0 1 0]
b:  [0 1 1 0 1 1 0 1]
pred:  [0 1 1 0 1 1 1 1]
true:  [1 1 0 0 1 1 1 1]
98 + 109 = 111
iter:  61000
smooth_loss:  [[4.06699416]]
a:  [0 1 0 0 0 0 1 0]
b:  [0 0 0 0 1 0 0 1]
pred:  [0 1 1 0 1 0 1 1]
true:  [0 1 0 0 1 0 1 1]
66 + 9 = 107
iter:  62000
smooth_loss:  [[4.07418318]]
a:  [0 0 1 1 0 1 1 1]
b:  [0 1 1 0 1 0 1 1]
pred:  [0 1 1 1 